In [144]:
import polars as pl
import datetime as dt
import numpy as np

In [145]:
def compute_weights(covariance, expected_returns): #This works
    w = np.linalg.solve(covariance, expected_returns)
    return w/np.sum(w)

def weights_error(cov_est, cov_real, expected_returns): #This definitely doesn't work
    return np.linalg.norm(compute_weights(cov_est, expected_returns) - compute_weights(cov_real, expected_returns))


In [147]:
start_date = dt.date(1996,7,3) #min value 1996,7,3
end_date = dt.date(2024, 12, 31) #max value 2024, 12, 31

bab = pl.read_parquet("signal_bab.parquet") #1
meanrev = pl.read_parquet("signal_meanrev.parquet") #2
momentum = pl.read_parquet("signal_momentum.parquet") #3

expected_returns_data = (
    bab.join(meanrev, on="date", how="inner")
       .join(momentum, on="date", how="inner")
).select(
    pl.col("date").alias("date"),
    pl.col("bab_return").rolling_mean(45).alias("expected_returns1"),
    pl.col("meanrev_return").rolling_mean(45).alias("expected_returns2"),
    pl.col("momentum_return").rolling_mean(45).alias("expected_returns3")
).filter(pl.col("date").is_between(start_date, end_date))

weekday = list(map(lambda date : date.weekday(), bab.filter(pl.col("date").is_between(start_date, end_date)).select(pl.col("date").alias("date")).to_series().to_list())) 

returns1 = bab.filter(pl.col("date").is_between(start_date, end_date)).select(pl.col("bab_return").alias("return1")).to_numpy()
returns2 = meanrev.filter(pl.col("date").is_between(start_date, end_date)).select(pl.col("meanrev_return").alias("return2")).to_numpy()
returns3 = momentum.filter(pl.col("date").is_between(start_date, end_date)).select(pl.col("momentum_return").alias("return3")).to_numpy()

len = returns1.size

returns = np.column_stack((np.ravel(returns1), 
                           np.ravel(returns2), 
                           np.ravel(returns3)))

expected_returns = expected_returns_data.select(
    pl.col("expected_returns1").alias("expected_returns1"),
    pl.col("expected_returns2").alias("expected_returns2"),
    pl.col("expected_returns3").alias("expected_returns3")
).to_numpy()

dif_returns = expected_returns - returns

outer_product = [(np.outer((dif_returns[i]), (dif_returns[i]))) for i in range(0, len)] 
#I had to rename this variable because it didn't mean what I thought it meant.
#I don't know what to call this so I've named it outer product.

def recurse(x_new, x_prev, day, r) : #Converges to the geometrically weighted mean.
    if (day == 0): return (1-r)*x_new + r*x_prev #If the day is monday, we use an alternative formula because weekends are non-trading days.
    else: return (1-r)*x_new + r*x_prev

r = 0.96 #After 120 days, the contribution of lower terms should be negligable. I just chose r so that r^120 < 0.01

#Remember, the estimated covariance matrix is the expected covariance matrix for the next day.
cov_matrix_est = [outer_product[0]]
for i in range(1, len-1):
    cov_matrix_est.append(recurse(outer_product[i], cov_matrix_est[i-1], weekday[i], r))

#This is the estimated covariance matrix for each day
#Make sure to estimate the covariance matrix for atleast 120 days. 
#This will insure that the junk starting value will have a negligible effect on the estimated covariance matrix.

print(cov_matrix_est)

#Use the compute_weights function to get the signal weights
#1 -> bab
#2 -> meanrev
#3 -> momentum

[array([[0.00036679, 0.00044989, 0.00019473],
       [0.00044989, 0.00055181, 0.00023885],
       [0.00019473, 0.00023885, 0.00010338]]), array([[0.00036289, 0.00046717, 0.00019405],
       [0.00046717, 0.00064527, 0.00025258],
       [0.00019405, 0.00025258, 0.00010394]]), array([[3.48849560e-04, 4.44562761e-04, 1.86400389e-04],
       [4.44562761e-04, 6.51822934e-04, 2.41572639e-04],
       [1.86400389e-04, 2.41572639e-04, 9.98104231e-05]]), array([[3.35157999e-04, 4.25416902e-04, 1.78250578e-04],
       [4.25416902e-04, 6.32832956e-04, 2.35514177e-04],
       [1.78250578e-04, 2.35514177e-04, 9.76522749e-05]]), array([[0.00034436, 0.00042572, 0.00019992],
       [0.00042572, 0.00062079, 0.00024815],
       [0.00019992, 0.00024815, 0.00013042]]), array([[0.00033162, 0.00041097, 0.00019433],
       [0.00041097, 0.00060097, 0.00024355],
       [0.00019433, 0.00024355, 0.00013084]]), array([[0.00034191, 0.00042715, 0.00022258],
       [0.00042715, 0.00062209, 0.00028368],
       [0.00022